In [16]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
from xml.etree import ElementTree as ET
from statsmodels.stats.multitest import multipletests

In [17]:
#negative similar region vs positive similar region
non_mhc_complementary_region={'ACB': {},
 'All': {},
 'ASW': {},
 'BEB': {'split_chr3/xav': [[98046342, 98273693]]},
 'CDX': {'split_chr13/xai': [[55102733, 55443491]]},
 'CEU': {'split_chr17/xaj': [[45670177, 46418024]]},
 'CHB': {},
 'CHS': {'split_chr3/xav': [[98046342, 98273693]]},
 'CLM': {'split_chr17/xaj': [[45836662, 46224960]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'ESN': {'split_chr5/xai': [[34264948, 34562837]]},
 'FIN': {},
 'GBR': {'split_chr1/xbt': [[226978066, 227352188]],
  'split_chr17/xaj': [[45670177, 46418024]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'GIH': {'split_chr3/xav': [[98046342, 98276757]]},
 'GWD': {},
 'IBS': {'split_chr17/xaj': [[45670177, 46418024]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'ITU': {'split_chr17/xak': [[52575391, 52889013]]},
 'JPT': {},
 'KHV': {},
 'LWK': {},
 'MSL': {'split_chr5/xai': [[34264948, 34562837]]},
 'MXL': {'split_chr1/xbt': [[227016487, 227352188]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'PEL': {'split_chr11/xaf': [[23168664, 23564494]],
  'split_chr3/xav': [[98046342, 98276757]]},
 'PJL': {},
 'PUR': {'split_chr17/xaj': [[45836662, 46418024]],
  'split_chr3/xav': [[98046342, 98276757]]},
 'STU': {'split_chr13/xai': [[55102733, 55443491]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'TSI': {'split_chr17/xaj': [[45670177, 46418024]]},
 'YRI': {}}

non_mhc_similarity_region={'ACB': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16000187, 18181463]],
  'split_chr2/xav': [[91720904, 94530414]],
  'split_chr3/xau': [[90103657, 93851087]]},
  'All':{'split_chr13_xaa': [[16000187, 18181463]],
 'split_chr1_xba': [[121579777, 123992934]],
 'split_chr1_xbb': [[123992935, 124938490]],
 'split_chr2_xav': [[91720904, 94530414]],
 'split_chr3_xau': [[90362505, 91477491]]},
 'ASW': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121635735, 123992934]],
  'split_chr13/xaa': [[16040220, 18181463]],
  'split_chr16/xaj': [[35510197, 46630361]],
  'split_chr2/xav': [[91720904, 94530414]],
  'split_chr3/xau': [[90103657, 93851087]]},
 'BEB': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16165134, 17417031]],
  'split_chr16/xaj': [[35510197, 46692526]],
  'split_chr2/xav': [[91992563, 94195617]]},
 'CDX': {},
 'CEU': {'split_chr1/xbb': [[123992935, 124920289]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16446009, 18181463]],
  'split_chr2/xav': [[92076199, 94195617]]},
 'CHB': {},
 'CHS': {'split_chr1/xba': [[123102017, 123978193]]},
 'CLM': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16446009, 18181463]],
  'split_chr16/xaj': [[35510197, 46630361]],
  'split_chr16/xai': [[32030198, 35510168]],
  'split_chr2/xav': [[91992563, 94654240]]},
 'ESN': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121635735, 123992934]],
  'split_chr13/xaa': [[16000187, 18181463]],
  'split_chr2/xav': [[91720904, 94654240]],
  'split_chr3/xau': [[90103657, 93851087]]},
 'FIN': {'split_chr1/xbb': [[124048603, 124877211]],
  'split_chr1/xba': [[121635735, 123992934]],
  'split_chr2/xav': [[91992563, 94195617]]},
 'GBR': {'split_chr1/xbb': [[123992935, 124877211]],
  'split_chr1/xba': [[122678788, 123992934]]},
 'GIH': {'split_chr1/xbb': [[123992935, 124877211]],
  'split_chr1/xba': [[121635735, 123992934]],
  'split_chr16/xaj': [[35510197, 46692526]],
  'split_chr2/xav': [[91951194, 94195617]]},
 'GWD': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16000187, 18181463]],
  'split_chr16/xaj': [[35510197, 46630361]],
  'split_chr2/xav': [[91720904, 94195617]],
  'split_chr3/xau': [[90103657, 93851087]]},
 'IBS': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16446009, 18024589]],
  'split_chr16/xaj': [[35510197, 46630361]],
  'split_chr2/xav': [[91992563, 94654240]]},
 'ITU': {'split_chr1/xbb': [[123992935, 124877211]],
  'split_chr1/xba': [[121635735, 123992934]],
  'split_chr16/xaj': [[35510197, 46692526]],
  'split_chr2/xav': [[91992563, 94195617]]},
 'JPT': {},
 'KHV': {},
 'LWK': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16040220, 18181463]],
  'split_chr2/xav': [[91720904, 94654240]],
  'split_chr3/xau': [[90103657, 93851087]]},
 'MSL': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121635735, 123992934]],
  'split_chr13/xaa': [[16040220, 18181463]],
  'split_chr2/xav': [[91720904, 94195617]],
  'split_chr3/xau': [[90103657, 93851087]]},
 'MXL': {'split_chr1/xbb': [[123992935, 124920289]],
  'split_chr1/xba': [[121635735, 123992934]],
  'split_chr16/xaj': [[35510197, 46692526]],
  'split_chr2/xav': [[91720904, 94735664]]},
 'PEL': {'split_chr1/xbb': [[123992935, 124782972]],
  'split_chr1/xba': [[122885884, 123992934]]},
 'PJL': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16165134, 18024589]],
  'split_chr16/xaj': [[35510197, 46692526]],
  'split_chr2/xav': [[91768349, 94195617]]},
 'PUR': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr13/xaa': [[16152209, 18181463]],
  'split_chr16/xaj': [[35510197, 46630361]],
  'split_chr2/xav': [[91720904, 94809756]]},
 'STU': {'split_chr1/xbb': [[123992935, 124920289]],
  'split_chr1/xba': [[121635735, 123992934]],
  'split_chr16/xaj': [[35510197, 46692526]],
  'split_chr16/xai': [[33059491, 35510168]],
  'split_chr2/xav': [[91992563, 94195617]]},
 'TSI': {'split_chr1/xbb': [[123992935, 124920289]],
  'split_chr1/xba': [[121579777, 123992934]],
  'split_chr2/xav': [[91992563, 94195617]]},
 'YRI': {'split_chr1/xbb': [[123992935, 124938490]],
  'split_chr1/xba': [[121635735, 123992934]],
  'split_chr13/xaa': [[16000187, 18181463]],
  'split_chr2/xav': [[91720904, 94530414]],
  'split_chr3/xau': [[90103657, 93851087]]}}

In [18]:
fileName = '../../1000_population.tsv'


lines = []
categories = []
samplenames=[]

with open(fileName, 'r') as f:
    text = f.read()

lines = text.split('\n')
for l in lines:
    samplenames.append(l.split('\t')[0])
    categories.append(l.split('\t')[-1])
allcategories=sorted(list(set(categories)))


In [19]:
# temp_non_mhc_complementary_region = non_mhc_complementary_region.copy()
# temp_non_mhc_similarity_region=non_mhc_similarity_region.copy()

In [20]:
# for race, region_dict in temp_non_mhc_complementary_region.items():

#     if region_dict=={}:
#         del non_mhc_similarity_region[race]
#         del non_mhc_complementary_region[race]
# for race, region_dict in temp_non_mhc_similarity_region.items():

#     if region_dict=={}:
#         del non_mhc_complementary_region[race]
#         del non_mhc_similarity_region[race]

In [21]:
split_repeat_region=dict()
for a in allcategories:
    split_repeat_region[a]=dict()

In [22]:
def find_overlap(arr1, arr2):
    overlap = set()  # 使用集合来存储重合部分

    for interval1 in arr1:
        for interval2 in arr2:
            start = max(interval1[0], interval2[0])
            end = min(interval1[1], interval2[1])
            if start <= end:
                overlap.add((start, end))  # 将重合部分添加到集合中

    return list(overlap)  # 将集合转换回列表形式




In [23]:

for race, region_dict in non_mhc_similarity_region.items():
    print("race",race)
    for split_sim,region_arrs_sim in region_dict.items():        
        if split_sim in non_mhc_complementary_region[race]:
            print("split",split_sim)
            region_arrs_com=non_mhc_complementary_region[race][split_sim]
            split_repeat_region[race][split_sim]=find_overlap(region_arrs_sim,region_arrs_com)
        else:
            continue



race ACB
race All
race ASW
race BEB
race CDX
race CEU
race CHB
race CHS
race CLM
race ESN
race FIN
race GBR
race GIH
race GWD
race IBS
race ITU
race JPT
race KHV
race LWK
race MSL
race MXL
race PEL
race PJL
race PUR
race STU
race TSI
race YRI


In [24]:
split_repeat_region

{'ACB': {},
 'ASW': {},
 'BEB': {},
 'CDX': {},
 'CEU': {},
 'CHB': {},
 'CHS': {},
 'CLM': {},
 'ESN': {},
 'FIN': {},
 'GBR': {},
 'GIH': {},
 'GWD': {},
 'IBS': {},
 'ITU': {},
 'JPT': {},
 'KHV': {},
 'LWK': {},
 'MSL': {},
 'MXL': {},
 'PEL': {},
 'PJL': {},
 'PUR': {},
 'STU': {},
 'TSI': {},
 'YRI': {}}